In [58]:
import pandas as pd

In [59]:
df = pd.read_csv(
    "/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/2000-2016/allegations.csv", low_memory=False)
profile_df = pd.read_csv(
    "/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/2000-2016/officer_profile.csv", low_memory=False)
# Drop all allegations that are not Use Of Force, Excessive Force
df = df[(df['Category'] == 'Use Of Force') | (df['Category'] == 'Excessive Force')]
# Drop unused columns
df = df.drop(["AllegationCode", "Latitude", "Longitude", "Location", "Add1", "Add2", "City", "RecommendedFinding", "RecommendedOutcome", "FinalFinding", "FinalOutcome", "Finding", "Outcome", "InvestigatorName", "InvestigatorRank", "StartDate", "EndDate"], axis=1)

In [60]:
df.columns

Index(['CRID', 'OfficerID', 'OfficeFirst', 'OfficerLast', 'Category',
       'Allegation', 'Beat', 'IncidentDate'],
      dtype='object')

In [61]:
df.head()

,CRID,OfficerID,OfficeFirst,OfficerLast,Category,Allegation,Beat,IncidentDate
20,C259003,6587,David,Deja,Use Of Force,Excessive Force / On Duty - No Injury,NaN,2000-01-01 00:00:00
24,C259010,13689,Charles,Jones,Use Of Force,Excessive Force / Off Duty - No Injury,NaN,2000-01-01 00:00:00
25,C259010,12961,Keith,Irvin,Use Of Force,Excessive Force / Off Duty - No Injury,NaN,2000-01-01 00:00:00
26,C259011,10864,Mark,Grohovena,Use Of Force,Excessive Force / Off Duty - No Injury,NaN,2000-01-01 00:00:00
29,C259013,3055,Cornelius,Brown,Use Of Force,Excessive Force / On Duty - Injury,NaN,2000-01-01 00:00:00


In [62]:
df.shape

(26720, 8)

In [63]:
# Note Complaint Register ID Number (CRID)
allegations_officers_df = df[["OfficerID","OfficeFirst","OfficerLast"]]
allegations_officers_df.head()

,OfficerID,OfficeFirst,OfficerLast
20,6587,David,Deja
24,13689,Charles,Jones
25,12961,Keith,Irvin
26,10864,Mark,Grohovena
29,3055,Cornelius,Brown


In [64]:
officers = pd.unique(allegations_officers_df["OfficerID"]).shape

In [65]:
pd.unique(df["Beat"]).shape

(275,)

In [66]:
profile_df.columns

Index(['OfficerID', 'OfficerFirst', 'OfficerLast', 'Gender', 'Race',
       'ApptDate', 'Unit', 'Rank', 'Star', 'Age'],
      dtype='object')

In [67]:
officers_2 = pd.unique(profile_df["OfficerID"]).shape

In [68]:
unique_officers = profile_df.drop_duplicates(subset="OfficerID")
unique_officers.shape

(16368, 10)

In [69]:
# Make a new column in unique_officers called "Beat" where the value is a float
unique_officers["Beat"] = 0
unique_officers["Beat"] = unique_officers["Beat"].astype(float)
# unique_officers["Beat"] = 0.0
unique_officers.columns

/var/folders/64/7cc975tn74sb3dg4ztck8zsw0000gn/T/ipykernel_37908/2406453936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_officers["Beat"] = 0
/var/folders/64/7cc975tn74sb3dg4ztck8zsw0000gn/T/ipykernel_37908/2406453936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_officers["Beat"] = unique_officers["Beat"].astype(float)


Index(['OfficerID', 'OfficerFirst', 'OfficerLast', 'Gender', 'Race',
       'ApptDate', 'Unit', 'Rank', 'Star', 'Age', 'Beat'],
      dtype='object')

In [71]:
unique_officers

,OfficerID,OfficerFirst,OfficerLast,Gender,Race,ApptDate,Unit,Rank,Star,Age,Beat
0,4,Carmel,Abbate,M,White,25209.0,NaN,Po As Detective,NaN,80,0.0
1,13,Dale,Abbott,M,White,30284.0,NaN,Police Officer,NaN,69,0.0
2,17,Moulay,Abdullah,M,Black,32868.0,NaN,Police Officer,NaN,64,0.0
3,19,Restituto,Abejero,M,Asian/Pacific,36507.0,NaN,Police Officer,NaN,73,0.0
4,34,Floyd,Abron,M,Black,35975.0,NaN,Police Officer,NaN,49,0.0
...,...,...,...,...,...,...,...,...,...,...,...
73789,31666,Agnieszka,Zieba,F,White,41338.0,NaN,Police Officer,NaN,38,0.0
73794,31750,Alen,Zulfic,M,White,41786.0,NaN,Police Officer,NaN,38,0.0
73925,32362,Jun,Sun,F,Asian/Pacific,37404.0,NaN,NaN,NaN,50,0.0
73951,33793,Cherie,Maureaux,F,White,NaN,NaN,NaN,NaN,51,0.0


In [77]:
for index, row in unique_officers.iterrows():
  # check if officer in df, if not then skip
  if row["OfficerID"] not in pd.unique(df["OfficerID"]):
    continue
  unique_officers.loc[index, "Beat"] = df.loc[df["OfficerID"] == row["OfficerID"], "Beat"].iloc[0]

In [78]:
# Load Excel Sheets using ExcelFile
zero = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2000_allegation.xlsx')
one = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis//Data/Website/2001_allegation.xlsx')
two = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2002_allegation.xlsx')
three = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2003_allegation.xlsx')
four = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2004_allegation.xlsx')
five = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2005_allegation.xlsx')
six = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2006_allegation.xlsx')
seven = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2007_allegation.xlsx')
eight = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2008_allegation.xlsx')
nine = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2009_allegation.xlsx')
ten = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2010_allegation.xlsx')
eleven = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis//Data/Website/2011_allegation.xlsx')
twelve = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2012_allegation.xlsx')
thirteen = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2013_allegation.xlsx')
fourteen = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2014_allegation.xlsx')
fifteen = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2015_allegation.xlsx')
sixteen = pd.ExcelFile('/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Website/2016_allegation.xlsx')

In [79]:
# Select excel sheet allegations from excel file
allegations_00 = zero.parse('Allegations')
allegations_01 = one.parse('Allegations')
allegations_02 = two.parse('Allegations')
allegations_03 = three.parse('Allegations')
allegations_04 = four.parse('Allegations')
allegations_05 = five.parse('Allegations')
allegations_06 = six.parse('Allegations')
allegations_07 = seven.parse('Allegations')
allegations_08 = eight.parse('Allegations')
allegations_09 = nine.parse('Allegations')
allegations_10 = ten.parse('Allegations')
allegations_11 = eleven.parse('Allegations')
allegations_12 = twelve.parse('Allegations')
allegations_13 = thirteen.parse('Allegations')
allegations_14 = fourteen.parse('Allegations')
allegations_15 = fifteen.parse('Allegations')
allegations_16 = sixteen.parse('Allegations')

# combine allegations from 2010-2016 into one csv
allegations = pd.concat([allegations_00, allegations_01, allegations_02, allegations_03, allegations_04, allegations_05, allegations_06, allegations_07, allegations_08, allegations_09, allegations_10, allegations_11, allegations_12, allegations_13,
                        allegations_14, allegations_15, allegations_16], ignore_index=True)

In [80]:
allegations_count = allegations.groupby('OfficerID').size().reset_index(name='allegations')
print(allegations_count.head())
unique_officers['num_allegations'] = unique_officers['OfficerID'].map(allegations_count.set_index('OfficerID')['allegations'])

   OfficerID  allegations
0          1            6
1          2            8
2          4            2
3          6            4
4          7            5


/var/folders/64/7cc975tn74sb3dg4ztck8zsw0000gn/T/ipykernel_37908/3749225212.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_officers['num_allegations'] = unique_officers['OfficerID'].map(allegations_count.set_index('OfficerID')['allegations'])


In [81]:
unique_officers.shape

(16368, 12)

In [82]:
unique_officers = unique_officers.dropna(subset=["Beat"])
unique_officers.shape

(14340, 12)

In [86]:
# if Beat = 0.0, then drop row
unique_officers = unique_officers[unique_officers.Beat != 0.0]
unique_officers.shape

(6771, 12)

In [87]:
print(unique_officers.head())

    OfficerID OfficerFirst OfficerLast Gender           Race  ApptDate  Unit  \
1          13         Dale      Abbott      M          White   30284.0   NaN   
5          38      Abdalla    Abuzanat      M  Asian/Pacific   35975.0   NaN   
7          58       Manuel     Acevedo      M       Hispanic   30151.0   NaN   
9          61       Martin     Acevedo      M       Hispanic   36290.0   NaN   
16         90        Craig       Adams      M          Black   23550.0   NaN   

               Rank  Star  Age    Beat  num_allegations  
1    Police Officer   NaN   69  1655.0                3  
5   Po As Detective   NaN   51   813.0               13  
7    Police Officer   NaN   66  1023.0                6  
9    Police Officer   NaN   49  1032.0                8  
16   Police Officer   NaN   80   123.0                8  


In [88]:
unique_officers.to_csv("/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Processed/unique_officers.csv", index=False)